In [1]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк. Обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [2]:
df = pd.read_csv("../data/transactions.csv", nrows=1000000)
df.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [3]:
genders = pd.read_csv("../data/gender_train.csv")
genders.head()

,customer_id,gender
0,10928546,1
1,69348468,1
2,61009479,0
3,74045822,0
4,27979606,1


In [4]:
mccs = pd.read_csv("../data/tr_mcc_codes.csv", sep=';')
mccs.head()

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело


In [5]:
tr_types = pd.read_csv("../data/tr_types.csv", sep=';')
tr_types.head()

,tr_type,tr_description
0,3200,Плата за предоставление услуг посредством моби...
1,3210,Плата за предоставление отчета по счету карты ...
2,3800,Плата за обслуживание банковской карты (за пер...
3,4000,Плата за получение наличных в Сбербанке
4,4001,Плата за получение наличных в Сбербанке (в дру...


## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

Если мы должны найти долю типов транзакций, которые содержат __POS__ или __ATM__ от всех уникальных типов транзакций в выборке `sample`:

In [46]:
# 1
sample = df['tr_type'].sample(1000)

In [47]:
unique_types_in_sample = sample.unique()

In [48]:
len(sample)

1000

### Ответ к заданию 1

In [49]:
# 2
tr_types[tr_types['tr_type'].isin(unique_types_in_sample)]['tr_description']\
    .str.contains('POS|ATM').value_counts(normalize=True)

False    0.735294
True     0.264706
Name: tr_description, dtype: float64

Если же мы должны найти долю транзакций в выборке `sample`, у которых в описании типа есть слова __POS__ или __ATM__ относительно всех 1000 транзакций в выборке, то:

In [50]:
types_contain_pos = tr_types[tr_types['tr_description'].str.contains('POS')]['tr_type']

### Ответ к заданию 1

In [53]:
# 2
sample.isin(types_contain_pos).value_counts()  # нормализировать не имеет смысла, сумма = 1000

False    624
True     376
Name: tr_type, dtype: int64

###### Примечание: ответы будут меняться при каждом новом `sample`

## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости.

In [58]:
# 1
df['tr_type'].value_counts()

1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
         ...  
2446         4
4096         4
8146         3
1510         2
8100         2
Name: tr_type, Length: 72, dtype: int64

In [60]:
# 2
df['tr_type'].value_counts()[:10]

1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
2370     49830
7010     28972
7030     26078
7071     15781
1100     15476
Name: tr_type, dtype: int64

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите разницу для этих клиентов между суммой расходов и суммой приходов

In [76]:
money_sums = df.groupby('customer_id').sum()

In [78]:
# 1
money_sums[money_sums['amount'] == max(money_sums['amount'])]

,mcc_code,tr_type,amount
customer_id,,,
95481247,4227905,2282858,50508877.7


In [79]:
# 2
money_sums[money_sums['amount'] == min(money_sums['amount'])]

,mcc_code,tr_type,amount
customer_id,,,
54871257,15954177,7081907,-801026035.2


Если нам надо найти разницу между `amount` у самого "богатого" и `amount` у самого "бедного", то:

In [111]:
round(float(money_sums[money_sums['amount'] == min(money_sums['amount'])]['amount']) - \
    float(money_sums[money_sums['amount'] == max(money_sums['amount'])]['amount']), 1)

-851534912.9

Если же нам надо найти разницу между всей прибылью и всей убылью у каждого из этих двух клиентов (`customer_id` 95481247 и `customer_id` 54871257), то:

In [90]:
richest = money_sums[money_sums['amount'] == max(money_sums['amount'])].index[0]

In [91]:
poorest = money_sums[money_sums['amount'] == min(money_sums['amount'])].index[0]

In [104]:
trs_of_richest = df[df['customer_id'] == richest]  # transactions of the richest customer

In [103]:
trs_of_poorest = df[df['customer_id'] == poorest]  # transactions of the poorest customer

In [112]:
# 3 (for customer_id = 95481247)
trs_of_richest[trs_of_richest['amount'] < 0]['amount'].sum() - \
    trs_of_richest[trs_of_richest['amount'] > 0]['amount'].sum()

-122309451.36

In [113]:
# 3 (for customer_id = 54871257)
trs_of_poorest[trs_of_poorest['amount'] < 0]['amount'].sum() - \
    trs_of_poorest[trs_of_poorest['amount'] > 0]['amount'].sum()

-1145257382.16

## Задание 4
1. Найдите среднее арифметическое и медиану по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по всем типам транзакций для клиентов из задания 3

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 транзакций для мужчин и для женщин, наименьших среди всех транзакций по полученным значениям "max_income". 
2. Выделите среди них те, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности для в тратах для разных полов.

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте ночные поступления для мужчин и женщин. Ночные - это примерно 00-06 часов. 